In [5]:
import os
import numpy as np
import time
import tensorflow as tf
from keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix, accuracy_score
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [6]:
# Path to model and labels
model_path = 'keras_model.h5'
labels_path = 'labels.txt'

# Load the trained model
model = load_model(model_path)

TypeError: Error when deserializing class 'DepthwiseConv2D' using config={'name': 'expanded_conv_depthwise', 'trainable': True, 'dtype': 'float32', 'kernel_size': [3, 3], 'strides': [1, 1], 'padding': 'same', 'data_format': 'channels_last', 'dilation_rate': [1, 1], 'groups': 1, 'activation': 'linear', 'use_bias': False, 'bias_initializer': {'class_name': 'Zeros', 'config': {}}, 'bias_regularizer': None, 'activity_regularizer': None, 'bias_constraint': None, 'depth_multiplier': 1, 'depthwise_initializer': {'class_name': 'VarianceScaling', 'config': {'scale': 1, 'mode': 'fan_avg', 'distribution': 'uniform', 'seed': None}}, 'depthwise_regularizer': None, 'depthwise_constraint': None}.

Exception encountered: Unrecognized keyword arguments passed to DepthwiseConv2D: {'groups': 1}

In [ ]:
with open(labels_path, 'r') as f:
    labels = f.read().splitlines()

In [ ]:
test_dir = {
    'crack': 'crack/',
    'good': 'good/',
    'pothole': 'pothole/'
}

In [ ]:
img_height, img_width = 224, 224  # Assuming model input size is 224x224
batch_size = 32

In [ ]:
y_true = []
y_pred = []

In [ ]:
datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
start_time = time.time()

for label, directory in test_dir.items():
    # Create a generator for test images
    test_generator = datagen.flow_from_directory(
        directory,
        target_size=(img_height, img_width),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False
    )
    
    # Predict using the model
    preds = model.predict(test_generator)
    
    # Get predicted classes and append results
    y_pred_classes = np.argmax(preds, axis=1)
    y_pred.extend(y_pred_classes)
    
    # Add true labels
    true_label_index = labels.index(label)
    y_true.extend([true_label_index] * len(y_pred_classes))

end_time = time.time()
classification_time = end_time - start_time

In [ ]:
accuracy = accuracy_score(y_true, y_pred)

In [ ]:
conf_matrix = confusion_matrix(y_true, y_pred)

In [ ]:
results_df = pd.DataFrame(conf_matrix, index=labels, columns=labels)

In [ ]:
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=labels, yticklabels=labels)
plt.title('Confusion Matrix')
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.show()

In [ ]:
# Print the accuracy and classification time
print(f"Classification Time: {classification_time:.2f} seconds")
print(f"Accuracy: {accuracy * 100:.2f}%")